<a href="https://colab.research.google.com/github/dey-hritam/ML-Projects/blob/main/Multiagent_Architecture_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers torch beautifulsoup4 requests pandas gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [6]:
!pip install -U langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [7]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import requests
from langchain_community.retrievers import TavilySearchAPIRetriever
import os
import pandas as pd
import numpy as np
from typing import List, Dict
import re
import gradio as gr

In [8]:
class ResearchAgent:
    def __init__(self):
        print("Loading GPT-2 model...")
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.model = GPT2LMHeadModel.from_pretrained('gpt2')
        self.model.eval()
        print("Model loaded successfully!")

    def generate_text(self, prompt: str, max_length: int = 150) -> str:
        # Encode the prompt
        inputs = self.tokenizer.encode(prompt, return_tensors='pt', truncation=True)
        with torch.no_grad():
            outputs = self.model.generate(
                inputs,
                max_length=max_length,
                num_return_sequences=1,
                temperature=0.7,
                pad_token_id=self.tokenizer.eos_token_id
            )
            # Decode the generated output
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return generated_text

    def clean_text(self, text: str) -> str:
        # Remove extra whitespace and normalize text
        text = re.sub(r'\s+', ' ', text).strip()
        return text




In [11]:
os.environ['TAVILY_API_KEY'] = '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'

In [12]:
class MarketResearchSystem:
    def __init__(self):
        self.agent = ResearchAgent()
        self.tavily_client = TavilySearchAPIRetriever(api_key=os.environ['TAVILY_API_KEY'])

    def search_web(self, query: str, num_results: int = 3) -> list:
        try:
            search_results = self.tavily_client.search(query, num_results=num_results)
            return [result.text for result in search_results]
        except Exception as e:
            print(f"Error in web search: {str(e)}")
            return []

    def analyze_industry(self, company_name: str) -> dict:
        research_prompt = f"Analyze the industry and market position of {company_name}. Consider their main products, services, and competitive advantages."
        ai_use_cases_prompt = f"List potential AI and machine learning use cases for {company_name} to improve their business operations and customer experience."

        industry_analysis = self.agent.generate_text(research_prompt)
        ai_use_cases = self.agent.generate_text(ai_use_cases_prompt)

        # Search for relevant datasets using Tavily
        datasets = self.search_web(f"{company_name} datasets")

        return {
            'company_name': company_name,
            'industry_analysis': self.agent.clean_text(industry_analysis),
            'ai_use_cases': self.agent.clean_text(ai_use_cases),
            'relevant_datasets': datasets
        }

In [13]:
def format_output(results: dict) -> str:
    output = f"""
    Company Analysis Report for {results['company_name']}
    ================================================

    Industry Analysis:
    -----------------
    {results['industry_analysis']}

    Potential AI Use Cases:
    ----------------------
    {results['ai_use_cases']}

    Relevant Datasets:
    -----------------
    """ + "\n".join(f"- {dataset}" for dataset in results['relevant_datasets'])

    return output

# Create the analysis system
system = MarketResearchSystem()

Loading GPT-2 model...
Model loaded successfully!


In [14]:
# Define Gradio Interface function

def analyze_company(company_name: str) -> str:
    if not company_name:
        return "Please enter a company name."

    try:
        results = system.analyze_industry(company_name)
        return format_output(results)
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Create and launch the Gradio interface
iface = gr.Interface(
    fn=analyze_company,
    inputs=gr.Textbox(label="Enter Company Name"),
    outputs=gr.Textbox(label="Analysis Results", lines=20),
    title="AI Market Research & Use Case Generator",
    description="Generate AI/ML use cases and market analysis for any company using GPT-2 and Tavily",
    examples=[["Microsoft"], ["Tesla"], ["Amazon"]]
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b2bcb23dc66b95695c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
